# DS Technical Mock Interview

In this project I will show how to build a simple Linear Regression model from data about used car prices from 2017. I will clean pieces of the data and demonstrate a simple feature engineering task. 

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np

Data is available here on Kaggle: https://www.kaggle.com/brentpafford/true-car-listings-2017-project?select=true_car_project_full.csv

In [2]:
df = pd.read_csv('true_car_project_full.csv')
df.head()

,Id,Price,Year,Mileage,City,State,City State,Vin,Make,Model,Region
0,1,16472,2015,18681,Jefferson City,MO,Jefferson City MO,KL4CJBSBXFB267643,Buick,EncoreConvenience,Midwest
1,2,15749,2015,27592,Highland,IN,Highland IN,KL4CJASB5FB245057,Buick,EncoreFWD,Midwest
2,3,16998,2015,13650,Boone,NC,Boone NC,KL4CJCSB0FB264921,Buick,EncoreLeather,Southeast
3,4,15777,2015,25195,New Orleans,LA,New Orleans LA,KL4CJASB4FB217542,Buick,EncoreFWD,Southeast
4,5,16784,2015,22800,Las Vegas,NV,Las Vegas NV,KL4CJBSB3FB166881,Buick,EncoreConvenience,Rockies


In [3]:
df.shape

(1048575, 11)

## Preprocessing

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Id          1048575 non-null  int64 
 1   Price       1048575 non-null  int64 
 2   Year        1048575 non-null  int64 
 3   Mileage     1048575 non-null  int64 
 4   City        1048575 non-null  object
 5   State       1048575 non-null  object
 6   City State  1048575 non-null  object
 7   Vin         1048575 non-null  object
 8   Make        1048575 non-null  object
 9   Model       1048575 non-null  object
 10  Region      1048575 non-null  object
dtypes: int64(4), object(7)
memory usage: 88.0+ MB


Create a new column that is the mixture of make and model with an underscore between them. Call this new column "Make_Model"

In [5]:
# simple combining of columns
df['Make_Model'] = df['Make'] + '_' + df['Model']

# Show that the change is correct
df.head()

,Id,Price,Year,Mileage,City,State,City State,Vin,Make,Model,Region,Make_Model
0,1,16472,2015,18681,Jefferson City,MO,Jefferson City MO,KL4CJBSBXFB267643,Buick,EncoreConvenience,Midwest,Buick_EncoreConvenience
1,2,15749,2015,27592,Highland,IN,Highland IN,KL4CJASB5FB245057,Buick,EncoreFWD,Midwest,Buick_EncoreFWD
2,3,16998,2015,13650,Boone,NC,Boone NC,KL4CJCSB0FB264921,Buick,EncoreLeather,Southeast,Buick_EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,New Orleans LA,KL4CJASB4FB217542,Buick,EncoreFWD,Southeast,Buick_EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,Las Vegas NV,KL4CJBSB3FB166881,Buick,EncoreConvenience,Rockies,Buick_EncoreConvenience


Drop the City, State, City State, Vin, Make, and Model columns

In [6]:
# remove the unnecessary columns
df = df.drop(columns={'City', 'State', 'City State', 'Vin', 'Make', 'Model'})

# Check to make sure that the columns that needed to be dropped are gone
df.head()

,Id,Price,Year,Mileage,Region,Make_Model
0,1,16472,2015,18681,Midwest,Buick_EncoreConvenience
1,2,15749,2015,27592,Midwest,Buick_EncoreFWD
2,3,16998,2015,13650,Southeast,Buick_EncoreLeather
3,4,15777,2015,25195,Southeast,Buick_EncoreFWD
4,5,16784,2015,22800,Rockies,Buick_EncoreConvenience


In [7]:
#Check for NAs
df.isna().sum()

Id            0
Price         0
Year          0
Mileage       0
Region        0
Make_Model    0
dtype: int64

In [8]:
#Check for duplicatess
duplicate = df[df.duplicated()]
duplicate

,Id,Price,Year,Mileage,Region,Make_Model


## ML

Here we will make a small linear regression model to predict the price of a used car. There are a lot of different Make_Model combinations in the data, so we will be using only the 30 most frequent of these. 

In [9]:
#Get a list of the 30 car make_models that were most frequent
top_30 = [name for name in df.Make_Model.value_counts().head(30).index]

Here we will create a new version of the dataframe. Take only the rows where the Make_Model is in top_30 

In [10]:
df = df.loc[df["Make_Model"].isin(top_30), :]

Get dummy variables for the categorical variables.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283292 entries, 681 to 1048570
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Id          283292 non-null  int64 
 1   Price       283292 non-null  int64 
 2   Year        283292 non-null  int64 
 3   Mileage     283292 non-null  int64 
 4   Region      283292 non-null  object
 5   Make_Model  283292 non-null  object
dtypes: int64(4), object(2)
memory usage: 15.1+ MB


In [13]:
# Binary encoding using Pandas (multiple columns)
df_binary_encoded = pd.get_dummies(df, columns=['Region', 'Make_Model'], drop_first=True, dtype='int')

In [15]:
# look at the data to ensure that the One-Hot encoding is right
df_binary_encoded.head()

,Id,Price,Year,Mileage,Region_Northeast,Region_Pacific,Region_Rockies,Region_Southeast,Region_Southwest,Make_Model_BMW_5,...,Make_Model_Hyundai_Santa,Make_Model_Hyundai_Sonata2.4L,Make_Model_Jeep_Grand,Make_Model_Jeep_Wrangler,Make_Model_Kia_OptimaLX,Make_Model_Kia_SorentoLX,Make_Model_Land_Rover,Make_Model_Lexus_RX,Make_Model_MINI_Cooper,Make_Model_Nissan_Altima2.5
681,682,8989,2009,70509,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
682,683,17750,2014,33026,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
683,684,18000,2014,49669,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
684,685,16888,2014,49877,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
686,687,8450,2008,96236,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Please explain what the following code chunk does. 

In [16]:
# Creating the scaler instance
data_scaler = StandardScaler()
# Fitting the scaler
data_scaler.fit(df_binary_encoded)
# Transforming the data
df_data_scaled = data_scaler.transform(df_binary_encoded)

The above cell scales the data to make the mean of each column 0 and the standard deviation to 1. The way it works is by subracting the mean from the original value then dividing that by the standard deviation of the column. When done to all the columns this can improve model performance by ensuring that all features contribute to the model equally. This is important because if some features have very large numbers they can dominate the machine learning algorithm and fail to capture the true relationships between features and target.

In [17]:
X = df_binary_encoded.iloc[:, 2:]
y = df_binary_encoded.iloc[:, 1]

Do a train/test split, with 20% test data.

In [18]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=12345)


Train a linear regression model with the data and get predictions.

In [20]:
# Instantiate linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

#Get predictions 
y_pred = model.predict(X_test)

In [21]:
mean_squared_error(y_pred, y_test)

32717205.841555245

In [22]:
import math
math.sqrt(mean_squared_error(y_pred, y_test))

5719.895614568088

Please describe what MSE is. Also please interpret the sqrt of the MSE.

The MSE - Mean Squared Error is the sqauared average error distance from the regression line. This shows the difference between the predicted values and the actual values. 

We take the square root of that in the Root Mean Squared Error - RMSE - in order to return to the original units of the target column. In this data set the target column is the value of each car in dollars since this data was taken from US. 

So the average error of predicted values is $5720 from the actual values. This is a pretty significantly high error and further exploration would be a good idea. It would be helpful to explore if there are any outliers that are informing the high error. It would also be a good idea to attempt to build alternative regression models that might perform more effectively at finding the values of the vehicles.